### gemma-2b-it Instruction Tuning
preprocess에서 전처리한 데이터를 이용해 gemma 모델을 학습시킵니다.
코드들은 [Stanford alpaca](https://github.com/tatsu-lab/stanford_alpaca)를 기반으로 작성되었습니다.
학습 환경은 아래와 같습니다.

|구분|내용|
|-|-|
|학습환경|Google Colab|
|GPU|L4(22.5GB)|
|VRAM|약 17GB 소요|
|dtype|bfloat16|
|Attention|flash attention2|
|Tuning|Lora(r=4, alpha=32)|
|Learning Rate|1e-4|
|Optimizer|adamw_torch_fused|

---

1. Pretrained 모델 설정  
GPU 환경에서는 [flash attention](https://github.com/Dao-AILab/flash-attention)을 사용하는 것이 좋습니다.
device는 cuda로 설정, dtype은 bfloat16으로 설정하였습니다.

2. 토크나이저를 정의합니다.  
추후 작성할 프롬프트들을 토크나이징하여 input_ids와 labels를 생성합니다.  

3. 마스킹
프롬프트들을 포맷팅한 다음 토크나이징 작업을 수행합니다.  
학습에 사용할 데이터들을 마스킹 작업하여 훈련에 사용할 수 있게 합니다.

4. `Dataset`과 `data_collator`를 정의합니다.  
2번의 토크나이징한 데이터로 `Dataset`을 생성하여 `train_data_set`으로 설정합니다.
`data_collator`를 통해 Sequence의 Padding을 수행합니다.

5. Lora 설정
Peft 학습을 위해 LoraConfig를 설정합니다.

6. 학습
1번의 Pretrained 모델, 4번의 Dataset 및 data_collator, 5번의 LoraConfig를 토대로 학습을 수행합니다.  

7. 기존 모델과 비교  
기존 gemma 모델과 Instruction Tuning된 모델의 답변을 비교해봅니다.

In [1]:
!pip install --quiet\
selenium==4.20.0\
datasets==2.19.0\
accelerate==0.27.2\
flash-attn==0.2.4\
peft==0.10.0\
trl==0.8.6\
transformers==4.40.1\
python-dotenv==1.0.1\
wandb==0.16.6\
evals==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB

In [2]:
import os
import sys
import json
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
import torch
from datasets import Dataset
from google.colab import userdata, drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/instruction-tuning-with-rag-example-main"

sys.path.append(path)
import utils
import prompts

Mounted at /content/drive


### 1. Pretrained 모델 설정
[google/gemma-1.1-2b-it](https://huggingface.co/google/gemma-2b-it)  
구글 Colab Secret 기능으로 허깅페이스 토큰을 사용할 수 있습니다.([Colab Secret 가이드](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75))  
gemma 모델은 토큰이 없으면 받을 수 없습니다.  

학습시 메모리 사용량을 크게 줄일 수 있는 방법은 3가지 정도였습니다.
+ Quantization
+ Flash Attention
+ PEFT

#### Quantization
`torch.bfloat16` 또는 `torch.float16` 타입으로 퍼포먼스를 비교적 떨어뜨리지 않으면서 적은 메모리를 사용할 수 있습니다.  
더 자세한 내용은 [huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/performance#floating-data-types)를 참고하세요.  

#### Flash Attention
GPU를 사용하는 환경이면 flash attention을 사용하길 추천드립니다.  
inference시 15GB정도의 VRAM이 소요되었지만, flash attention을 사용시 5GB 정도로 VRAM 사용량이 감소하였습니다.  
flash attention은 속도와 메모리 사용량을 감소시킬 수 있지만 퍼포먼스의 큰 차이가 없습니다.  
자세한 내용은 [https://arxiv.org/pdf/2307.0869](https://arxiv.org/pdf/2307.0869)을 참조해주세요.

#### PEFT
가장 많이 사용되는 LoRA의 경우 논문에 따르면 GPU 사용량이 1/3 가량으로 절감되었다고 합니다.  
또한 Full Fine Tuning과 비교하여 퍼포먼스의 차이도 크지 않다고 합니다(어떤 경우는 퍼포먼스가 더 좋았다고 합니다).  
자세한 내용은 [https://arxiv.org/pdf/2106.09685](https://arxiv.org/pdf/2106.09685)를 참조해주세요.

In [3]:
model_id = "google/gemma-1.1-2b-it"
dtype = torch.bfloat16
token = userdata.get('HF_TOKEN_READ') # Colab Secret에 설정 가능
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
    token=token,
    attn_implementation="flash_attention_2"
)
# 토크나이저의 max_length가 매우 큰수로 지정되어있으므로, 모델의 sequence length인 8192로 세팅
tokenizer.model_max_length = model.config.max_position_embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

### 2. tokenizer 설정
프롬프트들을 받아 Embedding index를 반환하는 토크나이저 기능을 정의합니다.

In [4]:
def tokenize(strings, tokenizer):
    tokenized_list = [
        tokenizer(
            text,
            max_length=tokenizer.model_max_length,
            return_tensors="pt",
            padding="longest",
            truncation=True
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

### 3. 마스킹
데이터셋은 아래의 예시와 같이 전달됩니다.  
+ `input_ids`
```python
tensor([  28693, 238264, 236648, 234541,  90621, 235248, 235284, 237029,  47555,
        235265, 171066,  90621, 150483, 236800, 238597,  74715, 239618, 236137,
          ...
         95917,  96564,  75500, 237433, 235248, 235284, 237936, 237699,  31087,
         96564,  83160, 237036, 149735, 179694, 235265,    108,      1])
```

+ `labels`
```python
tensor([  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          ...
         95917,  96564,  75500, 237433, 235248, 235284, 237936, 237699,  31087,
         96564,  83160, 237036, 149735, 179694, 235265,    108,      1])
```

Stanford Alpaca의 예시를 영문으로 변경하면 아래와 같은 형태입니다.  
+ `input_ids`

```plain
</s>Below is an instruction that describes a task. Write a response that appropriately completes the request.

Instruction:
Give three tips for staying healthy.

Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables.
2. Exercise regularly to keep your body active and strong.
3. Get enough sleep and maintain a consistent sleep schedule.</s>
```


+ `labels`

```plain
<MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK>
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.</s>
```
`<MASK>`는 임의로 적어둔 것이며, 실제로는 `input_ids`나 `labels`처럼 -100 값으로 채워지게 됩니다.  
-100은 Pytorch의 모든 Loss 함수에서 `ignore_index`로 사용되는 값이므로, Loss 계산시 제외됩니다.  
따라서 `input_ids`는 모든 Sequence, `labels`는 앞쪽은 마스킹되고 연산에 사용되는 Sequence는 ChatGPT의 답변 뿐입니다.

In [ ]:
# 파이토치에서 Loss 함수들의 ignore_index 값들은 모두 -100으로 처리되어 있습니다
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html
IGNORE_INDEX = -100

def preprocess(
    sources,
    examples,
    tokenizer,
):
    """ 프롬프트 데이터를 받아서 전처리"""
    examples_tokenized, sources_tokenized = [tokenize(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

### 4. `dataset`과 `collator` 정의
+ 데이터셋을 로드하여 프롬프트에 포맷팅합니다.
+ 포맷팅한 시퀀스를 `tokenizer`로 인코딩 및 마스킹 처리합니다.
+ 인코딩 및 마스킹 처리가 된 데이터를 `Dataset` 객체로 정의합니다.
+ batch 데이터를 처리하기 위해 `data_collator`를 정의합니다.
+ 데이터셋과 `data_collator`를 `trainer`에 전달하기 쉽게 `dict` 객체로 처리합니다.

In [ ]:
from torch.utils.data import Dataset

class SupervisedDataset(Dataset):
    """데이터셋을 불러오고 프롬프트에 포맷팅합니다. 만든 데이터셋은 Dataset객체로 생성합니다."""
    def __init__(self, tokenizer, data_path):
        super().__init__()
        instructions = utils.jload(data_path)
        sources = []
        examples = []
        for idx in range(len(instructions)):
            question = json.loads(instructions[idx])['question']
            answer = json.loads(instructions[idx])['answer']
            source = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer="")
            example = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer=answer) + tokenizer.eos_token
            sources.append(source)
            examples.append(example)

        data_dict = preprocess(sources, examples, tokenizer)
        self.input_ids = data_dict['input_ids']
        self.labels = data_dict['labels']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i):
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


`Dataset`은 huggingface의 Dataset 클래스를 사용하는 방법도 있습니다.  
아래 코드 역시 `train_dataset`을 생성하여 datasets.Dataset 객체로 생성합니다.  
`Dataset.from_dict` 메서드를 사용하면 별도의 클래스 설정 없이 간단하게 `Dataset`을 생성합니다.  

```python
def make_train_dataset(data_path: str):
    """데이터셋을 불러오고 프롬프트에 포맷팅합니다. 만든 데이터셋은 Dataset객체로 생성합니다."""

    instructions = utils.jload(data_path)
    sources = []
    examples = []
    for idx in range(len(instructions)):
        question = json.loads(instructions[idx])['question']
        answer = json.loads(instructions[idx])['answer']
        source = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer="")
        example = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer=answer) + tokenizer.eos_token
        sources.append(source)
        examples.append(example)

    data_dict = preprocess(sources, examples, tokenizer)
    train_dataset = Dataset.from_dict(data_dict)
    train_dataset.set_format("torch")

    return train_dataset
```

In [ ]:
class DataCollatorForSupervisedDataset(object):
    """데이터 콜레이터"""

    def __init__(self, tokenizer):
        self.tokenizer=tokenizer

    def __call__(self, instances):
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids_ = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels_ = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids_,
            labels=labels_,
            attention_mask=input_ids_.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
def make_supervised_data_module(data_path: str, tokenizer):
    """학습에 사용할 데이터셋과 콜레이터를 정의합니다."""
    train_dataset = SupervisedDataset(data_path=data_path, tokenizer=tokenizer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

### 5. LoRA PEFT
메모리 절감을 최대화 하기 위해 모델 내 Adapter를 추가할 수 있는 모든 Layer에 LoRA를 적용하였습니다.
+ Layer이름은 모델마다 다르니 모델의 Layer를 확인하고 추가해주세요.
+ `TrainingArguments`의 경우, 여러번의 학습을 통해 적절한 하이퍼파라미터를 찾아서 학습시켰습니다.  

아래 코드는 [huggingface의 블로그](https://huggingface.co/blog/gemma-pefthttps://huggingface.co/blog/gemma-peft)를 참조하였습니다.

파라미터에 대한 설명은 huggingface에서 확인해주세요.  
+ [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)
+ [LoraConfig](https://huggingface.co/docs/peft/package_reference/lora#peft.LoraConfig)
+ [SFTTRainer](https://huggingface.co/docs/trl/sft_trainer#trl.SFTTrainer)  

파라미터를 수정하면서 어떻게 모델이 훈련되는지 지켜보는 것도 좋은 방법입니다.  
또한 파라미터를 수정해보면서 GPU에 어떤 영향을 주는지 보는 것도 좋습니다.  


In [ ]:
from transformers import TrainingArguments

my_model_name = os.path.join(path, "gemma-2b-it-example-v1")

args = TrainingArguments(
    output_dir=my_model_name,
    num_train_epochs=10,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=5e-5,
    max_grad_norm=0.3,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    bf16=True,
    push_to_hub=True,
    hub_token=userdata.get("HF_TOKEN"),
    report_to="wandb",
    run_name="gemma-2b-it-example-v1"
)

In [ ]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.0,
        r=4,
        bias="none",
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

### 6. 학습

wandb 사용법은 [wandb guide](https://docs.wandb.ai/guides/integrations/huggingface) 페이지를 참조해주세요.

In [ ]:
import wandb

wandb.login()

wandb: Currently logged in as: sk8terbo2. Use `wandb login --relogin` to force relogin


True

In [ ]:
from trl import SFTTrainer

data_path = os.path.join(path, "data/instruction.jsonl")
max_seq_length = model.config.max_position_embeddings
data_module = make_supervised_data_module(data_path=data_path, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    **data_module,
)
trainer.train()
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.398800
20,2.321900
30,2.087200
40,1.755300
50,1.665200
60,1.622000
70,1.551200
80,1.508000
90,1.436700
100,1.410900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

### 7. Gemma 기본 모델과 비교

In [10]:
# google/gemma-1.1-2b-it
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    token=userdata.get("HF_TOKEN"),
    attn_implementation="flash_attention_2"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# 학습한 파인튜닝 모델
finetuned_model = AutoModelForCausalLM.from_pretrained(
    "aiqwe/gemma-2b-it-example-v1",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
from utils import generate

rag_config = {
    "api_client_id": userdata.get('NAVER_API_ID'),
    "api_client_secret": userdata.get('NAVER_API_SECRET')
    }

🚀 기존 Gemma Instruction 모델

In [20]:
query = "전세사기에 대처하는 방법"

In [21]:
# 훈련 전 모델
completion = generate(
    model=model,
    tokenizer=tokenizer,
    query=query,
    max_new_tokens=512,
    rag=True,
    rag_config=rag_config
)
print(completion)


<bos><start_of_turn>user
전세사기에 대처하는 방법
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
-깡통전세 감별기 사기 대처하는 방법 총 정리 부동산 경기의 침체로 시장이 혼란스러운 가운데, 비교적 경험이 적은 신호부부나 사회 초년생을 대상으로 하는 전세 및 월세 사기가 기승을 부리고 있습니다.... 
 -'전세사기를 당했을 때 전세보증금 돌려받는 법'은 전세 사기 피해자들이 겪는 심리적, 경제적 어려움에 공감하며 이에 대처하는 방법을 안내한다. 저자들은 증거 수집과 변호사 상담의 중요성을 강조하고 보험금 청구와... 
 -전세사기에 대처하는 방법을 알아보기 전에, 전세사기가 무엇인지, 왜 우리가 주의해야 하는지에 대해 간략하게 설명해드리겠습니다. 전세사기는 집주인 또는 중개인이 세입자에게 전세금을 돌려주지 않거나, 실제로... 
 -전세사기대처하는방법 그러니 금액이 합리적이라고 해도실력에 대한 의심은하지 않으셔도 됩니다정확한 지식이 없는 상태에서는 제대로 판단하지 못하기 때문에 명확한 내용의 가이드를 초기에 받아보시는 것이... <end_of_turn>
<start_of_turn>model
전세사기에 대처하는 방법은 여러 가지가 있지만, 가장 중요한 것은 증거 수집과 변호사 상담이다. 또한 보험금 청구와 전세보증금 돌려받는 것 역시 매우 유용하다.

전세사기가 무엇인지, 그리고 우리가 주의해야 할까는 이해하기 위해서는 전세사기의 개념을 정확히 파악하는 것이 도움이 된다.<eos>


🚀 Instruction Tuning된 모델

In [22]:
completion = generate(
    model=my_model,
    tokenizer=tokenizer,
    query=query,
    repetition_penalty=1.5,
    temperature=0.5,
    max_new_tokens=512,
    rag=True,
    rag_config=rag_config
)
print(completion)


<bos><start_of_turn>user
전세사기에 대처하는 방법
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
-깡통전세 감별기 사기 대처하는 방법 총 정리 부동산 경기의 침체로 시장이 혼란스러운 가운데, 비교적 경험이 적은 신호부부나 사회 초년생을 대상으로 하는 전세 및 월세 사기가 기승을 부리고 있습니다.... 
 -'전세사기를 당했을 때 전세보증금 돌려받는 법'은 전세 사기 피해자들이 겪는 심리적, 경제적 어려움에 공감하며 이에 대처하는 방법을 안내한다. 저자들은 증거 수집과 변호사 상담의 중요성을 강조하고 보험금 청구와... 
 -전세사기에 대처하는 방법을 알아보기 전에, 전세사기가 무엇인지, 왜 우리가 주의해야 하는지에 대해 간략하게 설명해드리겠습니다. 전세사기는 집주인 또는 중개인이 세입자에게 전세금을 돌려주지 않거나, 실제로... 
 -전세사기대처하는방법 그러니 금액이 합리적이라고 해도실력에 대한 의심은하지 않으셔도 됩니다정확한 지식이 없는 상태에서는 제대로 판단하지 못하기 때문에 명확한 내용의 가이드를 초기에 받아보시는 것이... <end_of_turn>
<start_of_turn>model
**전세사기에 대처할 때 필요한 것들:** 전세보증금, 분양권 계약서, 사진、경찰의 입장서 등

전세사기가 발생하면 즉시 보험금을 청구하도록 노력하십시오. 또한, 전문변호사의 도움을 받고 증거를 확보하여 전세사기 대책을 마련하세요.
<eos>


In [23]:
# model.generate
input_text = "아파트 재건축에 대해 알려줘."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = finetuned_model.generate(**input_ids, max_new_tokens=512)
print(tokenizer.decode(outputs[0]))


<bos>아파트 재건축에 대해 알려줘.

**재건축에 관한 법률과 정책에 대해 알아보고, 재건축에 필요한 자금을 조달하는 방법을 알려주시는가?**

재건축에 관한 법률과 정책을 통해 재건축이 가능하며, 자금을 조달하는 방법으로는 재건축 자금조합의 설립, 조합원의 납부, 공공기금 등이 있습니다.

**재건축 프로젝트의 주요 단계를 알려주시는가?**

재건축 프로젝트의 주요 단계로는 사업시행계획서 작성, 사업시행단계, 사업시행계획수립인가, 사업시행, 재건축사업시행, 완성단계로 나눌 수 있습니다.

**재건축에 대한 더 자세한 정보를 원하는 경우, 국토교통부의 홈페이지에 접속하여 다양한 자료를 확인할 수 있습니다.**<eos>
